In [1]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from splinter import Browser
executable_path = {'executable_path': '/Users/jcson/chromedriver'}
browser = Browser("chrome", **executable_path, headless=False)

In [3]:
nasa_url = 'https://mars.nasa.gov/news/'
browser.visit(nasa_url)
time.sleep(1)
nasa_html = browser.html
nasa_soup = BeautifulSoup(nasa_html, 'html.parser')

list_nasa = nasa_soup.find('ul', class_='item_list')
first = list_nasa.find('li', class_='slide')
title = first.find('div', class_='content_title').text
summary = first.find('div', class_='article_teaser_body').text

In [4]:
jpl_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(jpl_url)
time.sleep(1)
browser.click_link_by_partial_text('FULL IMAGE')
time.sleep(1)
browser.click_link_by_partial_href('javascript:;')
time.sleep(1)

jpl_html = browser.html
jpl_soup = BeautifulSoup(jpl_html, 'html.parser')

img_relative = jpl_soup.find('img', class_='fancybox-image')['src']
image_path = f'https://www.jpl.nasa.gov{img_relative}'
print(image_path)

https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA18826_ip.jpg


In [5]:
weather_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(weather_url)
time.sleep(1)
weather_html = browser.html
weather_soup = BeautifulSoup(weather_html, 'html.parser')

tweets = weather_soup.find('ol', class_='stream-items')
weather = tweets.find('p', class_="tweet-text").text
print(weather)

A view of this morning’s launch of the Parker #SolarProbe launch from about 3 miles awayhttps://youtu.be/hDhIane1Rso 


In [6]:
facts_url = 'https://space-facts.com/mars/'
browser.visit(facts_url)
time.sleep(1)
facts_html = browser.html
facts_soup = BeautifulSoup(facts_html, 'html.parser')

fact_table = facts_soup.find('table', class_='tablepress tablepress-id-mars')
column1 = fact_table.find_all('td', class_='column-1')
column2 = fact_table.find_all('td', class_='column-2')

facets = []
values = []

for row in column1:
    facet = row.text.strip()
    facets.append(facet)
    
for row in column2:
    value = row.text.strip()
    values.append(value)
    
mars_facts = pd.DataFrame({
    "Facet":facets,
    "Value":values
    })

mars_facts_html = mars_facts.to_html(header=False, index=False)
mars_facts

,Facet,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [2]:
hemisphere_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
hemi_dicts = []

for i in range(1,9,2):
    hemi_dict = {}
    
    browser.visit(hemisphere_url)
    time.sleep(1)
    hemi_html = browser.html
    hemi_soup = BeautifulSoup(hemi_html, 'html.parser')
    hemi_name_links = hemi_soup.find_all('a', class_='product-item')
    hemi_name = hemi_name_links[i].text.strip('Enhanced')
    
    detail_links = browser.find_by_css('a.product-item')
    detail_links[i].click()
    time.sleep(1)
    browser.find_link_by_text('Sample').first.click()
    time.sleep(1)
    browser.windows.current = browser.windows[-1]
    img_html = browser.html
    browser.windows.current = browser.windows[0]
    browser.windows[-1].close()
    
    browser.click_link_by_partial_text('Open')
    
    img_soup = BeautifulSoup(img_html, 'html.parser')
    img_path = img_soup.find('img')['src']

    print(hemi_name)
    hemi_dict['title'] = hemi_name.strip()
    
    print(img_path)
    hemi_dict['img_url'] = img_path

    hemi_dicts.append(hemi_dict)

Cerberus Hemisphere 
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg
Schiaparelli Hemisphere 
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg
Syrtis Major Hemisphere 
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg
Valles Marineris Hemisphere 
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg
